In [1]:
import tensorflow as tf
import numpy as np

# Build and train a model

In [2]:
x = tf.placeholder(shape=[None, 2], dtype=tf.float32, name='my_input')
W = tf.get_variable(name='my_weight', shape=[2, 1], dtype=tf.float32)
y = tf.matmul(x, W)

W0801 16:13:30.522975  7296 deprecation.py:506] From f:\anaconda3\envs\tensorflow1.14\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [3]:
y_true = tf.placeholder(shape=[None, 1], dtype=tf.float32)
loss = tf.losses.mean_squared_error(labels=y_true, predictions=y)
train_op = tf.train.AdamOptimizer(learning_rate=0.01).minimize(loss)

W0801 16:13:30.669435  7296 deprecation.py:323] From f:\anaconda3\envs\tensorflow1.14\lib\site-packages\tensorflow\python\ops\losses\losses_impl.py:121: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [4]:
init_op = tf.global_variables_initializer()

sess = tf.Session()
sess.run(init_op)
for i in range(1000):
    x_train = np.random.rand(100, 2)
    y_train = np.sum(x_train, axis=1, keepdims=True)
    _, loss_value, w = sess.run([train_op, loss, W], feed_dict={x: x_train, y_true: y_train})
    if (i + 1) % 100 == 0:
        print('current loss: {}, current weights: {}'.format(loss_value, w))

current loss: 0.002988465130329132, current weights: [[0.8608876]
 [1.1213897]]
current loss: 6.662252872047247e-06, current weights: [[0.9931975]
 [1.0058186]]
current loss: 5.54492181337185e-11, current weights: [[0.99998343]
 [1.0000141 ]]
current loss: 9.756639804880604e-14, current weights: [[1.0000008]
 [0.9999994]]
current loss: 8.668232960844929e-14, current weights: [[1.0000008]
 [0.9999994]]
current loss: 9.066525470929204e-14, current weights: [[1.0000007]
 [0.9999994]]
current loss: 6.419614595768938e-14, current weights: [[1.0000007]
 [0.9999994]]
current loss: 8.283595977123109e-14, current weights: [[1.0000007]
 [0.9999994]]
current loss: 5.292322163190713e-14, current weights: [[1.0000006]
 [0.9999995]]
current loss: 4.6114221924527243e-14, current weights: [[1.0000006]
 [0.9999995]]


# Save model

https://www.tensorflow.org/guide/saved_model

* A <b>MetaGraph</b> is a dataflow graph, plus its associated variables, assets, and signatures.
* A `MetaGraphDef` is the protocol buffer representation of a MetaGraph.
* A <b>signature</b> is the set of inputs to and outputs from a graph.
* The `tf.saved_model.Builder` class provides functionality to save multiple `MetaGraphDef`s. Each `MetaGraphDef` added to the SavedModel must be annotated with user-specified `tag`s

In [5]:
export_dir = 'model'
saved_model =  tf.saved_model.Builder(export_dir)

https://www.tensorflow.org/tfx/serving/serving_basic

You can add meta graph and variables to the builder using `tf.saved_model.Builder.add_meta_graph_and_variables()` with the following arguments:
* `sess` is the TensorFlow session that holds the trained model you are exporting.
* `tags` is the set of tags with which to save the meta graph. In this case, since we intend to use the graph in serving, we use the `serve` tag from predefined SavedModel tag constants.
* `signature_def_map` specifies the map of user-supplied key for a <b>signature</b> to a tensorflow::SignatureDef to add to the meta graph. Signature specifies what type of model is being exported, and the input/output tensors to bind to when running inference.

  The special signature key serving_default specifies the default serving signature. The default serving signature def key, along with other constants related to signatures, are defined as part of SavedModel signature constants.

In [6]:
tensor_info_x = tf.saved_model.build_tensor_info(x)
tensor_info_y = tf.saved_model.build_tensor_info(y)

predict_signature = tf.saved_model.build_signature_def(
    inputs={'numbers': tensor_info_x},
    outputs={'sum': tensor_info_y},
    method_name=tf.saved_model.signature_constants.PREDICT_METHOD_NAME) # 'tensorflow/serving/predict'

W0801 16:13:31.335718  7296 deprecation.py:323] From <ipython-input-6-3a2a3f3ecd0e>:1: build_tensor_info (from tensorflow.python.saved_model.utils_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.


`tf.saved_model.Builder.save()` `as_text` argument:
* `False` (default): save as binary format (`.pb`)
* `True`: save as text format (`.pbtxt`)

In [7]:
saved_model.add_meta_graph_and_variables(
    sess=sess, 
    tags=[tf.saved_model.tag_constants.SERVING], # 'serve'
    signature_def_map={
        tf.saved_model.signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY: # 'serving_default'
        predict_signature
    }, 
    strip_default_attrs=True
)
saved_model.save(as_text=False)

sess.close() # close the session

# Inspect SavedModel

In [8]:
!saved_model_cli.exe show --dir model --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['numbers'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 2)
        name: my_input:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['sum'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 1)
        name: MatMul:0
  Method name is: tensorflow/serving/predict


In [9]:
!saved_model_cli.exe show --dir model --tag_set serve --signature_def serving_default

The given SavedModel SignatureDef contains the following input(s):
  inputs['numbers'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 2)
      name: my_input:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['sum'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 1)
      name: MatMul:0
Method name is: tensorflow/serving/predict


# Load model

In [10]:
tf.reset_default_graph() # reset graph
sess = tf.Session() # create a new session

In [11]:
meta_graph_def = tf.saved_model.load(sess=sess, 
                                     tags=[tf.saved_model.tag_constants.SERVING], # 'serve'
                                     export_dir=export_dir)
signature_def = \
    meta_graph_def.signature_def[tf.saved_model.signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY] # 'serving_default'

W0801 16:13:37.289789  7296 deprecation.py:323] From <ipython-input-11-a4b03a26711b>:3: load (from tensorflow.python.saved_model.loader_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
W0801 16:13:37.326083  7296 deprecation.py:323] From f:\anaconda3\envs\tensorflow1.14\lib\site-packages\tensorflow\python\training\saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


In [12]:
inputs = {}
for k in signature_def.inputs.keys():
    inputs[k] = tf.get_default_graph().get_tensor_by_name(signature_def.inputs[k].name)
inputs

{'numbers': <tf.Tensor 'my_input:0' shape=(?, 2) dtype=float32>}

In [13]:
outputs = {}
for k in signature_def.outputs.keys():
    outputs[k] = tf.get_default_graph().get_tensor_by_name(signature_def.outputs[k].name)
outputs

{'sum': <tf.Tensor 'MatMul:0' shape=(?, 1) dtype=float32>}

In [14]:
feed_dict = {
    inputs['numbers']: np.array([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
}
print(sess.run(outputs['sum'], feed_dict=feed_dict))
sess.close()

[[ 2.9999995]
 [ 7.       ]
 [11.       ]]
